# DTSC 580: Data Manipulation

## Assignment:  Halloween Candy

### Name: 

## Overview

In this assignment, your job will be to clean and wrangle data from a survey of Halloween candy to prepare it for a machine learning project. 

Once you have completed all the tasks in the notebook, save your notebook as `halloween_candy`, and verify that all the tests pass in CodeGrade.  

For those of you that are more comfortable with using Pandas, I have also provided extra optional data analysis questions at the end of the assignment for more practice.

## Data Set

The data set that we will be using is the 2017 Halloween Candy Hierarchy data set as discussed in this [boingboing](https://boingboing.net/2017/10/30/the-2017-halloween-candy-hiera.html) article.  You can also read more about the data in the [Science Creative Quarterly](https://www.scq.ubc.ca/so-much-candy-data-seriously/).

The following are the rating instructions from the survey:  

> Basically, consider that feeling you get when you receive this item in your Halloween haul. Does it make you really happy (JOY)? Or is it something that you automatically place in the junk pile (DESPAIR)? MEH for indifference, and you can leave blank if you have no idea what the item is.

Note that the original data set has been slightly altered from its original state, and if you wanted to perform any analysis for future projects, you would need to download the data directly from the links above.

This data is a great example of a messy data set, especially since they allowed respondents to enter text for a number of the fields. Also, note that some of the comments in the file might be considered inappropriate to some readers but cleaning this type of data is normal in a lot of data science projects.

## Note

<u>Show Work</u>

Remember that you must show your work.  Students submissions are spot checked manually throughout the term to verify that they are not hard coding the answer from looking only in the file or in CodeGrade's expected output.  If this is seen, the student's answer will be manually marked wrong and their grade will be changed to reflect this. 

For example, if the answer to Q1, the mean of a specific column, is 22:
```
# correct way
Q1 = df['column_name'].mean()

# incorrect way
Q1 = 22 
```

## Our End Goal

Our end goal for this project is to clean the data so that we could then create a machine learning model. We want to see if we are able to predict a person's gender based purely on their candy preferences. Although, you will not be creating a model for this assignment, only cleaning the data. The results of the models that I used after cleaning the data are provided at the end of this notebook.

## Initial Import & Exploration

In [1]:
# initial imports
import pandas as pd
import numpy as np

# Do not change this option; This allows the CodeGrade auto grading to function correctly
pd.set_option('display.max_columns', 20)

Let's start by importing our data and creating a DataFrame called `candy`.  We need to include `encoding='iso-8859-1'` during the import because there are special characters in the data that Pandas doesn't recognize. This happens a lot when attempting to import data where the public is able to input answers, especially if there are foreign language characters included. The normal encoding for Pandas is `utf-8`, so changing the encoding allows Pandas to recognize those special characters. 

Run the following code, with the encoding argument, and it should import correctly.

In [2]:
# read_csv with iso-8859-1 encoding; using latin-1 would also work here
candy_full = pd.read_csv('candy.csv', encoding='iso-8859-1')

# copy to new DF so that we can have a copy of the original import if needed
candy = candy_full.copy()

Let's take a brief look at the data by using `head()`.

In [3]:
# first five rows
candy.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,...,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
2,90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,...,NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,...,NaN,NaN,White and gold,NaN,Friday,NaN,1.0,NaN,NaN,"(70, 10)"


Next, run the following code to see information about the DataFrame.

In [4]:
# check info about the DataFrame
candy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479 entries, 0 to 2478
Columns: 120 entries, Internal ID to Click Coordinates (x, y)
dtypes: float64(4), int64(1), object(115)
memory usage: 2.3+ MB


Notice that this did not print the columns as you might be used to seeing. According to the Pandas documentation:  "If the DataFrame has more than max_cols columns, the truncated output is used. By default, the setting in pandas.options.display.max_info_columns is used." 

We can make the columns display by setting the `max_cols` argument equal to the number of columns in the data set.

In [5]:
# check info, set max_cols
candy.info(max_cols=120)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479 entries, 0 to 2478
Data columns (total 120 columns):
 #    Column                                                                                 Non-Null Count  Dtype  
---   ------                                                                                 --------------  -----  
 0    Internal ID                                                                            2479 non-null   int64  
 1    Q1: GOING OUT?                                                                         2368 non-null   object 
 2    Q2: GENDER                                                                             2437 non-null   object 
 3    Q3: AGE                                                                                2394 non-null   object 
 4    Q4: COUNTRY                                                                            2414 non-null   object 
 5    Q5: STATE, PROVINCE, COUNTY, ETC                                   

Of course, if you are just looking for the column names, you can just use a simple `for` loop.

In [6]:
# print a list of column names
for col in candy.columns:
    print(col)


Internal ID
Q1: GOING OUT?
Q2: GENDER
Q3: AGE
Q4: COUNTRY
Q5: STATE, PROVINCE, COUNTY, ETC
Q6 | 100 Grand Bar
Q6 | Anonymous brown globs that come in black and orange wrappers	(a.k.a. Mary Janes)
Q6 | Any full-sized candy bar
Q6 | Black Jacks
Q6 | Bonkers (the candy)
Q6 | Bonkers (the board game)
Q6 | Bottle Caps
Q6 | Box'o'Raisins
Q6 | Broken glow stick
Q6 | Butterfinger
Q6 | Cadbury Creme Eggs
Q6 | Candy Corn
Q6 | Candy that is clearly just the stuff given out for free at restaurants
Q6 | Caramellos
Q6 | Cash, or other forms of legal tender
Q6 | Chardonnay
Q6 | Chick-o-Sticks (we donÕt know what that is)
Q6 | Chiclets
Q6 | Coffee Crisp
Q6 | Creepy Religious comics/Chick Tracts
Q6 | Dental paraphenalia
Q6 | Dots
Q6 | Dove Bars
Q6 | Fuzzy Peaches
Q6 | Generic Brand Acetaminophen
Q6 | Glow sticks
Q6 | Goo Goo Clusters
Q6 | Good N' Plenty
Q6 | Gum from baseball cards
Q6 | Gummy Bears straight up
Q6 | Hard Candy
Q6 | Healthy Fruit
Q6 | Heath Bar
Q6 | Hershey's Dark Chocolate
Q6 | HersheyÕ

This data set is pretty messy. Your goal is now to perform the following actions to get it to the point where it can be passed to a machine learning model.

**Note: Unless the instructions ask you to do something different, please always update the original `candy` DataFrame for the exercises below.  The automatic grading in CodeGrade will check your final DataFrame and ensure that you have performed all required data manipulations.  Also, feel free to add additional cells as needed.**  

## Data Cleaning

**Exercise1:** Taking a look at the column names, you may notice that some include the character `Õ`. This should instead be an apostrophe `'` mark. Rename the column names that include the `Õ` character and replace it with an apostrophe.  

Remember that you should be updating the `candy` DataFrame for the tasks listed as unless told differently. 

In [7]:
candy.rename({"Q6 | Chick-o-Sticks (we donÕt know what that is)" : "Q6 | Chick-o-Sticks (we don't know what that is)"}, axis=1, inplace=True)
candy.rename({"Q6 | Peanut M&MÕs" : "Q6 | Peanut M&M's"}, axis=1, inplace=True)
candy.rename({"Q6 | ReeseÕs Peanut Butter Cups" : "Q6 | Reese's Peanut Butter Cups"}, axis=1, inplace=True)
candy.rename({"Q6 | HersheyÕs Milk Chocolate" : "Q6 | Hershey's Milk Chocolate"}, axis=1, inplace=True)


**Q1:** How many duplicated rows are there in the file? Assume that a duplicate is any row that is *exactly* the same as another one. Save this number as `Q1`.

In [8]:
Q1=candy.duplicated().sum()
Q1

17

**Q2:** How many duplicated rows are there in the file if we were to assume that a duplicate is any row with the same `Internal ID` number as another. In other words, even if the other values are different, a row would count as a duplicate if it had the same `Internal ID` as another. Save this number as `Q2`.

In [9]:
Q2=candy['Internal ID'].duplicated().sum()
Q2

19

**Exercise2:** Drop any duplicates from the `candy` DataFrame.  Duplicates are to be defined as any row with the same `Internal ID` as another. Use the default setting that keeps the first record from the duplicates.

In [10]:
ID=candy['Internal ID']
candy=candy.drop_duplicates('Internal ID')
candy['Internal ID'].duplicated().sum()

0

**Exercise3:** Your next task is to remove the following columns from the `candy` DataFrame as we will not use these columns for this project.  You are welcome to do further analysis on these columns but do not save your analysis in this notebook.

Remove the following columns: `Internal ID`, `Q5: STATE, PROVINCE, COUNTY, ETC`, `Q7: JOY OTHER`, `Q8: DESPAIR OTHER`, `Q9: OTHER COMMENTS`, `Unnamed: 113`, `Click Coordinates (x, y)`.

In [11]:
#df.drop(columns=['B', 'C'])
candy=candy.drop(columns=['Internal ID', 'Q5: STATE, PROVINCE, COUNTY, ETC',
                          'Q7: JOY OTHER', 'Q8: DESPAIR OTHER',
                          'Q9: OTHER COMMENTS', 'Unnamed: 113', 'Click Coordinates (x, y)'])


**Code Check:** As a check for the above exercises, the shape of your data should now be: `(2460, 113)`

In [12]:
candy.shape

(2460, 113)

**Exercise4:** Let's now take a look at the `Q2: GENDER` column since this will be what we are trying to predict. Take a look at the value counts for this column.

In [13]:
candy['Q2: GENDER'].value_counts()

Male                  1466
Female                 839
I'd rather not say      83
Other                   30
Name: Q2: GENDER, dtype: int64

**Q3:** How many missing values are in the `Q2: GENDER` column? Save this as `Q3`.

In [14]:
Q3= candy['Q2: GENDER'].isnull().sum()
Q3

42

**Exercise5:** Using the `candy` DataFrame, remove all rows with a missing value in the `Q2: GENDER` column.  (This should overwrite and be saved as `candy` like you have been doing for the previous exercises.)

In [15]:
#df.dropna(subset=['name', 'toy'])
candy=candy.dropna(subset=['Q2: GENDER'])
candy['Q2: GENDER'].isnull().sum()

0

**Exercise6:** For this project, we want to use binary classification, which predicts one of two classes. We want to predict between `Male` or `Female`. Because of this, select only the rows that contain either `Male` or `Female` in the `Q2: GENDER` column.

In [16]:
#array = ['yellow', 'green']
#df.loc[df['favorite_color'].isin(array)]
array = ['Male', 'Female']
candy=candy.loc[candy['Q2: GENDER'].isin(array)]
candy['Q2: GENDER'].value_counts()

Male      1466
Female     839
Name: Q2: GENDER, dtype: int64

**Code Check:** As a check for the above exercises, the shape of your data should now be: `(2305, 113)`

In [17]:
candy.shape

(2305, 113)

Now, let's work on filling some of the missing data.  There are easier ways to do this with the sklearn library which you will learn about more in the machine learning classes, but for now, let's try to practice our Pandas skills.

**Q4:** How many missing values are in the `Q1: GOING OUT?` column? Save this number as `Q4`.

In [18]:
Q4=candy['Q1: GOING OUT?'].isnull().sum()
Q4


77

**Exercise7:** For a future analysis question, we are interested in those that we know will *definitely* go out for Halloween.  Because of this, fill all missing values in the `Q1: GOING OUT?` column with a `No` value.

In [19]:
candy['Q1: GOING OUT?'].fillna('No', inplace=True)

**Code Check:** Double check your above work and look at the value counts for the `Q1: GOING OUT?` column.  Make sure that you only have "Yes" and No" values and that they add up to 2305, which is the number of rows you should have at this step in the assignment.

In [20]:
candy['Q1: GOING OUT?'].value_counts().sum()

2305

**Exercise8:** To get ready for the next step, let's practice selecting all the columns: going from `Q6 | 100 Grand Bar` to `Q11: DAY`.  Save this slice as `candy_slice`.

In [21]:
candy_slice=candy.iloc[:, 4:109]
candy_slice

,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,...,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q10: DRESS,Q11: DAY
1,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,...,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,...,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday
4,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,...,DESPAIR,JOY,MEH,JOY,JOY,DESPAIR,DESPAIR,JOY,White and gold,Friday
5,JOY,DESPAIR,JOY,NaN,NaN,NaN,MEH,MEH,DESPAIR,JOY,...,MEH,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,JOY,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2474,JOY,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,MEH,...,JOY,JOY,MEH,JOY,DESPAIR,MEH,DESPAIR,MEH,White and gold,Friday
2475,MEH,DESPAIR,JOY,NaN,NaN,NaN,NaN,DESPAIR,DESPAIR,JOY,...,DESPAIR,JOY,NaN,NaN,JOY,DESPAIR,MEH,JOY,Blue and black,Friday
2476,MEH,DESPAIR,JOY,DESPAIR,MEH,JOY,DESPAIR,MEH,MEH,DESPAIR,...,MEH,MEH,MEH,JOY,MEH,DESPAIR,DESPAIR,MEH,Blue and black,Friday
2477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Exercise9:** Now that you know how to slice the data, fill any missing values in the `candy` DataFrame for those columns (going from `Q6 | 100 Grand Bar` to `Q11: DAY`) with the string `NO_ANSWER`. Make sure you are working with the `candy` DataFrame and not the `candy_slice` DataFrame.

In [22]:
#test=candy.iloc[:, 4:109].fillna('NO_ANSWER')
#test=(candy.fillna[:, 4:109],('NO_ANSWER'))
candy.iloc[:, 4:109]=candy.iloc[:, 4:109].fillna('NO_ANSWER')


**Exercise10:** For all four `Q12: Media` columns in the `candy` DataFrame, fill the missing values with `0.0`.

In [23]:
candy.iloc[:, 109:114]=candy.iloc[:, 109:114].fillna(0.0)
candy.head()

,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),...,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q10: DRESS,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo]
1,No,Male,44,USA,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,...,DESPAIR,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday,0.0,1.0,0.0,0.0
2,No,Male,49,USA,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,0.0,0.0,0.0,0.0
3,No,Male,40,us,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,...,JOY,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday,0.0,1.0,0.0,0.0
4,No,Male,23,usa,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,...,JOY,DESPAIR,DESPAIR,JOY,White and gold,Friday,0.0,1.0,0.0,0.0
5,No,Male,NaN,NaN,JOY,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,NO_ANSWER,...,JOY,DESPAIR,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,0.0,1.0,0.0,0.0


**Code Check:** As a check for the above code, make sure that there are no missing values left for the `Q6` to `Q12` columns.  

In [24]:
candy.iloc[:, 4:114].value_counts().isnull().sum()

0

Now, let's look at the very messy `Q4: COUNTRY` column and see what we can do about it. First, run the code below to look at the different unique values in the data.

In [25]:
# check unique values
candy['Q4: COUNTRY'].unique()

array(['USA ', 'USA', 'us', 'usa', nan, 'canada', 'Canada', 'Us', 'US',
       'Murica', 'United States', 'uk', 'United Kingdom', 'united states',
       'Usa', 'United States ', 'United staes',
       'United States of America', 'UAE', 'England', 'UK', 'canada ',
       'United states', 'u.s.a.', '35', 'france',
       'United States of America ', 'america', 'U.S.A.', 'finland',
       'unhinged states', 'Mexico', 'Canada ', 'united states of america',
       'US of A', 'The United States', 'North Carolina ', 'Unied States',
       'Netherlands', 'germany', 'Europe', 'U S', 'u.s.', 'U.K. ',
       'Costa Rica', 'The United States of America', 'unite states',
       'U.S.', '46', 'Australia', 'Greece', 'USA? Hard to tell anymore..',
       "'merica", '45', 'United State', '32', 'France', 'australia',
       'Can', 'Canae', 'Trumpistan', 'Ireland', 'United Sates', 'Korea',
       'California', 'Unites States', 'Japan', 'USa', 'South africa',
       'I pretend to be from Canada, but I am

**Code Check:** As a check for the Country column, check to see how many unique values are in the data.  You should have `115` different unique values for the `Q4: COUNTRY` column.  If you have less or more than this number, double check your work above.

In [26]:
# check the Q4: COUNTRY number of unique values
candy['Q4: COUNTRY'].nunique()

115

We want to clean up this data to only include four areas: USA, Canada, Europe (the continent, not necessarily the European Union), and Other.

There are different ways to do this, but I would suggest that you look at the way we handled the `property_type` column in the `vienna` data set and the code in the `amenities_to_columns()` function in the module notebook.  These might be a little harder than those examples but they should give you a good baseline approach.  

You could use `replace()` for this step, and it is fine if you ultimately decide to do this, but I would suggest that you come up with a solution similar to what was shown in the `vienna` data cleaning notebook.  This method would be much more robust if you had many more values in your data.

I suggest the following order for this section to make it easier:
- Fill in all missing values with `Other`
- Code Australia as `Other` (doing this step will help when trying to use `us` in the next step if you use string methods)
- Combine all USA entries together as `USA`
- Combine Canadian entries as `CA`
- Combine European entries as `EU`
- Everything else gets coded as `Other`

**Exercise11:** Fill the missing values in the `Q4: Country` column with `Other`.

In [27]:
candy['Q4: COUNTRY'].fillna('Other', inplace=True)

**Code Check:** Double check that there are no missing values in the `Q4: COUNTRY` column.  Also, double check the unique values to make sure that "Other" was added.  This should mean that you now have `116` unique values for this column.

In [28]:
candy['Q4: COUNTRY'].value_counts().isnull().sum()

0

In [29]:
candy['Q4: COUNTRY'].nunique()

116

**Exercise12:** Combine all Australia entries into `Other`.  Watch out for capitalization issues.  You should have `114` unique values after this step.

In [30]:
#candy['Q4: COUNTRY']= candy['Q4: COUNTRY'].map(
                        #{'Australia' : 'Other'}
#df.replace(to_replace={'X': {5: 50}}, value=None)

candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Australia' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'australia' : 'Other'}}, value=None)

In [31]:
candy['Q4: COUNTRY'].nunique()

114

**Exercise13:** Combine all United States entries together into `USA`.  These would include the following:
```
'USA ', 'USA', 'us', 'usa', 'Us', 'US', 'Murica', 'United States', 'united states', 'Usa', 'United States ', 'United staes', 'United States of America', 'United states', 'u.s.a.', 'United States of America ', 'america', 'U.S.A.', 'unhinged states', 'united states of america', 'US of A', 'The United States', 'North Carolina ', 'Unied States', 'U S', 'u.s.', 'The United States of America', 'unite states','U.S.', 'USA? Hard to tell anymore..', "'merica", 'United State', 'United Sates', 'California', 'Unites States', 'USa', 'I pretend to be from Canada, but I am really from the United States.', 'Usa ', 'United Stated', 'New Jersey', 'United ststes', 'America', 'United Statss', 'murrika', 'USA! USA! USA!', 'USAA', 'united States ', 'N. America', 'USSA', 'U.S. ', 'u s a', 'United Statea', 'united ststes', 'USA USA USA!!!!'
```

In [32]:

candy=candy.replace(to_replace={'Q4: COUNTRY' : {'USA ' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'us' : 'USA'}}, value=None)  
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'usa' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Us' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'US' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Murica' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'United States' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'united states' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Usa' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'United States ' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'United staes' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'United States of America' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'United States of America ' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'United states' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'u.s.a.' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'america' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'U.S.A.' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'unhinged states' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'united states of america' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'US of A' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'The United States' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'North Carolina ' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Unied States' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'U S' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'u.s.' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'The United States of America' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'unite states' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'U.S.' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'USA? Hard to tell anymore..' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {"'merica" : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'United State' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'United Sates' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'California' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Unites States' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'USa' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'I pretend to be from Canada, but I am really from the United States.' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Usa ' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'United Stated' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'New Jersey' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'United ststes' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'America' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'United Statss' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'murrika' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'USA! USA! USA!' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'USAA' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'united States ' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'N. America' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'USSA' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'U.S. ' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'u s a' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'United Statea' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'united ststes' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'USA USA USA!!!!' : 'USA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'United states' : 'USA'}}, value=None)


**Code Check:** You should be merging the above values together into 1 (`USA`) and be left with 61 unique values after this step (including the `USA` value).

In [33]:
candy['Q4: COUNTRY'].nunique()

61

In [34]:
candy['Q4: COUNTRY'].value_counts()

USA                  1955
Canada                168
canada                 34
Other                  29
United Kingdom         13
                     ... 
UAE                     1
South africa            1
Canada`                 1
france                  1
Fear and Loathing       1
Name: Q4: COUNTRY, Length: 61, dtype: int64

**Exercise14:** Combine the Canadian entries (both upper and lower case) and label them as `CA`. Be careful as there are extra spaces, characters, and misspellings (Can, Canae). 

These values include:
```
'canada', 'Canada', 'canada ', 'Canada ', 'Can', 'Canae', 'Canada`', 'CANADA'
```

In [35]:
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'canada' : 'CA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Canada' : 'CA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'canada ' : 'CA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Can' : 'CA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Canae' : 'CA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Canada`' : 'CA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'CANADA' : 'CA'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Canada ' : 'CA'}}, value=None)


candy['Q4: COUNTRY'].value_counts()

USA                               1955
CA                                 216
Other                               29
United Kingdom                      13
UK                                  13
Germany                              7
Netherlands                          6
Japan                                5
Scotland                             4
Ireland                              3
Mexico                               3
germany                              3
Sweden                               2
China                                2
Switzerland                          2
Denmark                              2
Uk                                   2
uk                                   2
46                                   1
Ireland                              1
The Netherlands                      1
endland                              1
soviet canuckistan                   1
Singapore                            1
UAE                                  1
Taiwan                   

**Code Check:** You should be merging 8 values together into 1 (`CA`) and be left with 54 unique values after this step (including the `CA` value).

In [36]:
candy['Q4: COUNTRY'].nunique()

54

**Exercise15:** Combine the European entries and label them as `EU`. Again, we are looking at the continent of Europe and not necessarily the countries that are a part of the European Union.  

These values include:
```
'uk', 'United Kingdom', 'England', 'UK', 'france', 'finland', 'Netherlands', 'germany', 'Europe', 'U.K. ', 'Greece', 'France', 'Ireland', 'Uk', 'Germany', 'Scotland', 'UK ', 'Denmark', 'France ', 'Switzerland', 'Scotland ', 'The Netherlands', 'Ireland ', 'spain', 'Sweden', 'United kingdom'
```

In [37]:
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'uk' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'United Kingdom' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'England' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'UK' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'france' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'finland' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Netherlands' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'germany' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Europe' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'U.K. ' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Greece' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'France' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Ireland' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Uk' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Germany' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Scotland' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'UK ' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Denmark' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'France ' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Switzerland' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Scotland ' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'The Netherlands' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Ireland ' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'spain' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Sweden' : 'EU'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'United kingdom' : 'EU'}}, value=None)

**Code Check:** You should be merging 26 entries together and be left with 29 unique values after this step (including the `EU` value).

In [38]:
candy['Q4: COUNTRY'].nunique()

29

**Exercise16:** Finally, combine the other entries and label them as `Other`.

In [39]:
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Japan' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Mexico' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'China' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'subscribe to dm4uz3 on youtube' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Narnia' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Hong Kong' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Indonesia' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'hong kong' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Taiwan' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Singapore' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'I don\'t know anymore' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'soviet canuckistan' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'endland' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'South africa' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'South Korea' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'UD' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Korea' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Trumpistan' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'32' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'45' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'46' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Costa Rica' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'35' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'UAE' : 'Other'}}, value=None)
candy=candy.replace(to_replace={'Q4: COUNTRY' : {'Fear and Loathing' : 'Other'}}, value=None)
                                               

**Code Check:** Double check that you only have four unique values in the `Q4: COUNTRY` column: `USA`, `Other`, `CA`, and `EU`

In [40]:
candy['Q4: COUNTRY'].value_counts()

USA      1955
CA        216
EU         73
Other      61
Name: Q4: COUNTRY, dtype: int64

**Q5:** To double check that everything was coded correctly, save the value counts of the `Q4: COUNTRY` column as `Q5`.  You can check this once you run your CodeGrade check.

In [41]:
Q5=candy['Q4: COUNTRY'].value_counts()

We now want to look at the `Q3: AGE` column. Let's look at all the unique values.

In [42]:
candy['Q3: AGE'].value_counts()

34            88
37            86
43            85
40            85
36            78
              ..
45-55          1
Old enough     1
Many           1
no             1
60+            1
Name: Q3: AGE, Length: 95, dtype: int64

Again, this is a pretty messy column of data. This is a good example of why those that create online surveys shouldn't allow the individual to just put any value into the field. But it is now our job to clean this up.

**Exercise17:** Your task is to put these values into the following categorical bins: `unknown`, `17 and under`, `18-25`, `26-35`, `36-45`, `46-55`, and `56+`. 

- The category labels should exactly match the above.
- Missing values should be replaced with the `unknown` category
- To make things easier and avoid ambiguity, let's say that any value with text, even if we could determine the age, will be binned with the `unknown` category. For example: `sixty-nine` should be coded as `unknown`, `45-55` should be coded as `unknown`, `59 on the day after Halloween` should be coded as `unknown`, etc.
- Ensure that the category labels are unordered but reorder the categories so that 'unknown' is listed in the first position. This is not really needed but will help us grade your assignment. The categories should be listed as follows: `Index(['unknown', '17 and under', '18-25', '26-35', '36-45', '46-55', '56+'], dtype='object')`

First, we will replace any non-numeric value (those with text as mentioned above) with a missing value.  This will allow you to turn the other values into floats so that you can bin them. Just don't forget to code the missing values as `unknown` when you are done.  To replace the non-numeric values, run the following code:

In [43]:
# create True/False index
age_index = candy['Q3: AGE'].str.isnumeric()
# for the index, fill missing values with False
age_index = age_index.fillna(False)
# select Age column for only those False values from index and code as missing
candy.loc[~age_index, 'Q3: AGE'] = np.nan

In [44]:
#candy['Q3: AGE']=candy['Q3: AGE'].str.isnumeric()

#pd.cut(x=candy['Q3: AGE'], bins=[0, 17, 25, 35, 45, 55, 100])
#pd.cut(x=candy['Q3: AGE'], bins=[0, 17, 25, 35, 45, 55, 100],
                   #labels = ['17 and under', '18-25', '26-35',
                             #'36-45', '46-55', '56+'])
#candy['Q3: AGE']

**Exercise18:** Double check yourself by checking the categories for the `Q3: AGE` column. It should output: `Index(['unknown', '17 and under', '18-25', '26-35', '36-45', '46-55', '56+'], dtype='object')`

**Code Check:** To double check your above binning worked correctly, your value counts (sorted by the index) should be as follows:

```
unknown: 60 
17 and under: 49 
18-25: 85
26-35: 520
36-45: 768
46-55: 525
56+: 298
```

In [46]:
candy['Q3: AGE'].fillna('unknown', inplace=True)

You can also double check some of your work up to this point by making sure that there are no missing values in the data set anymore.

**Code Check:** Check to see if there are any missing values in the data set. Your output should show `0`.

In [47]:
candy.isnull().sum()

Q1: GOING OUT?             0
Q2: GENDER                 0
Q3: AGE                    0
Q4: COUNTRY                0
Q6 | 100 Grand Bar         0
                          ..
Q11: DAY                   0
Q12: MEDIA [Daily Dish]    0
Q12: MEDIA [Science]       0
Q12: MEDIA [ESPN]          0
Q12: MEDIA [Yahoo]         0
Length: 113, dtype: int64

**Exercise19:** Before you move on to the next section, reset the index for `candy` ensuring that it goes from 0 to n-1.  

In [48]:
candy.reset_index()

,index,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),...,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q10: DRESS,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo]
0,1,No,Male,44,USA,MEH,DESPAIR,JOY,MEH,DESPAIR,...,DESPAIR,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday,0.0,1.0,0.0,0.0
1,2,No,Male,49,USA,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,0.0,0.0,0.0,0.0
2,3,No,Male,40,USA,MEH,DESPAIR,JOY,MEH,MEH,...,JOY,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday,0.0,1.0,0.0,0.0
3,4,No,Male,23,USA,JOY,DESPAIR,JOY,DESPAIR,MEH,...,JOY,DESPAIR,DESPAIR,JOY,White and gold,Friday,0.0,1.0,0.0,0.0
4,5,No,Male,unknown,Other,JOY,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,...,JOY,DESPAIR,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300,2474,No,Male,24,USA,JOY,DESPAIR,MEH,DESPAIR,DESPAIR,...,DESPAIR,MEH,DESPAIR,MEH,White and gold,Friday,0.0,0.0,0.0,0.0
2301,2475,No,Female,33,USA,MEH,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,...,JOY,DESPAIR,MEH,JOY,Blue and black,Friday,0.0,1.0,0.0,0.0
2302,2476,No,Female,26,USA,MEH,DESPAIR,JOY,DESPAIR,MEH,...,MEH,DESPAIR,DESPAIR,MEH,Blue and black,Friday,0.0,1.0,0.0,0.0
2303,2477,No,Male,58,USA,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,0.0,0.0,0.0,0.0


## Feature Engineering 

Feature engineering is the process of transforming raw data into features that better represent the underlying problem to the predictive models.  In this section, we will create a new column called "net_feelies" (calculated by the authors as the total joy count minus the total despair count).

First, let's narrow down our data to make working with it easier.

**Exercise20:** Select only the Q6 candy columns (`Q6 | 100 Grand Bar` through `Q6 | York Peppermint Patties`) in the data set and save this as a new DataFrame called `candy_reduced`.

In [49]:
candy_reduced=candy.iloc[:,4:107]
#candy.iloc[:, 109:114]=candy.iloc[:, 109:114]


Next, we will create two Series, one with JOY counts and one with DESPAIR counts to add to our `candy_reduced` data.

**Exercise21:** Create a Series called `joy_count` that lists total counts for JOY for each column, making sure to keep it in the same order as the columns in the `candy_reduced` DataFrame. Hint: A simple way to do this is to filter the entire DataFrame for any `JOY` values and then use `count()`. See this [stackoverflow question](https://stackoverflow.com/questions/63103090/how-do-i-count-specific-values-across-multiple-columns-in-pandas) and answers.

In [50]:
joy_count=candy_reduced[candy_reduced == 'JOY'].count()

**Exercise22:** Same as above except you will create a Series called `despair_count` that lists the total counts for DESPAIR for each column.

In [51]:
despair_count=candy_reduced[candy_reduced == 'DESPAIR'].count()

**Exercise23:** Take the transpose of the `candy_reduced` DataFrame and save this transposed data as `candy_reduced_transpose`.

In [52]:
candy_reduced_transpose=candy_reduced.T

**Exercise24:** Add a new column called "joy_count" using the `joy_count` Series above and a new column called 'despair_count" using the `despair_count` Series above to the `candy_reduced_transpose` DataFrame.

In [53]:
candy_reduced_transpose['joy_count']=joy_count
candy_reduced_transpose['despair_count']=despair_count

**Exercise25:** Add a new column to the `candy_reduced_transpose` DataFrame called "net_feelies" that takes the `joy_count` column and subtracts the `despair_count` column.

In [54]:
minus=joy_count - despair_count
candy_reduced_transpose['net_feelies']=minus

**Exercise26:** Select only the `joy_count`, `despair_count`, and `net_feelies` columns from the `candy_reduced_transpose` DataFrame. Sort this DataFrame in descending order by `net_feelies` and save this as `candy_net_sorted`.

In [55]:
candy_net_sorted=candy_reduced_transpose[['joy_count', 'despair_count', 'net_feelies']]
candy_net_sorted=candy_net_sorted.sort_values('net_feelies', ascending=False)

## Encoding

We now want to get the `candy` DataFrame ready to run a machine learning algorthim to determine if we could predict a person's gender based on what candy they prefer.

You will learn more about this in the machine learning classes, but some algorithms work exclusively with numeric values. We will now turn all of our values into numeric values.  There are easier ways to do this with sklearn, which you will study in later courses, but we will use Pandas to perform these exercises for further practice.

**Exercise27:** For grading purposes, we want to leave the `candy` DataFrame as is. Make a copy of the `candy` DataFrame and save this new DataFrame as `candy_encode`.

In [56]:
candy_encode=candy.copy()

**Exercise28:** For the `candy_encode` DataFrame, replace any `Female` values with `0` and any `Male` values with `1`.

In [57]:
candy_encode.replace({'Female' : 0,
                     'Male' : 1})

,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),...,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q10: DRESS,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo]
1,No,1,44,USA,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,...,DESPAIR,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday,0.0,1.0,0.0,0.0
2,No,1,49,USA,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,0.0,0.0,0.0,0.0
3,No,1,40,USA,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,...,JOY,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday,0.0,1.0,0.0,0.0
4,No,1,23,USA,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,...,JOY,DESPAIR,DESPAIR,JOY,White and gold,Friday,0.0,1.0,0.0,0.0
5,No,1,unknown,Other,JOY,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,NO_ANSWER,...,JOY,DESPAIR,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2474,No,1,24,USA,JOY,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,...,DESPAIR,MEH,DESPAIR,MEH,White and gold,Friday,0.0,0.0,0.0,0.0
2475,No,0,33,USA,MEH,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,NO_ANSWER,...,JOY,DESPAIR,MEH,JOY,Blue and black,Friday,0.0,1.0,0.0,0.0
2476,No,0,26,USA,MEH,DESPAIR,JOY,DESPAIR,MEH,JOY,...,MEH,DESPAIR,DESPAIR,MEH,Blue and black,Friday,0.0,1.0,0.0,0.0
2477,No,1,58,USA,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,0.0,0.0,0.0,0.0


**Exercise29:** Again, you will learn more about this later, but we need to separate the column that we want to predict (called the response) and the columns that we will use to make the predictions (called the features).  **For both of the items below, make sure that the index is reset and goes from 0 to n-1.**

- Select only the `Q2: GENDER` column from `candy_encode` and save this as `candy_response`.  **Note: This should be a Series.**
- Drop the following columns from the `candy_encode` DataFrame: `Q2: GENDER`,`Q1: GOING OUT?`,`Q3: AGE`,`Q4: COUNTRY`,`Q10: DRESS`,`Q11: DAY`, `Q12: MEDIA [Daily Dish]`,`Q12: MEDIA [Science]`,`Q12: MEDIA [ESPN]`,`Q12: MEDIA [Yahoo]`.  Save the remaining columns as `candy_features`.

In [58]:
candy_response=candy_encode['Q2: GENDER']
candy_features=candy.drop(['Q1: GOING OUT?', 'Q2: GENDER', 'Q3: AGE',
                          'Q4: COUNTRY', 'Q10: DRESS', 'Q11: DAY',
                          'Q12: MEDIA [Daily Dish]', 'Q12: MEDIA [Science]',
                          'Q12: MEDIA [ESPN]', 'Q12: MEDIA [Yahoo]'], axis=1)


**Exercise30:** Use Panda's `get_dummies()` to encode the `candy_features` data, making sure to set `drop_first=True`. Save this as `candy_features_encoded`.

In [59]:
candy_features_encoded=pd.get_dummies(candy_features,drop_first=True)
candy_features_encoded

,Q6 | 100 Grand Bar_JOY,Q6 | 100 Grand Bar_MEH,Q6 | 100 Grand Bar_NO_ANSWER,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)_JOY,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)_MEH,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)_NO_ANSWER,Q6 | Any full-sized candy bar_JOY,Q6 | Any full-sized candy bar_MEH,Q6 | Any full-sized candy bar_NO_ANSWER,Q6 | Black Jacks_JOY,...,Q6 | Whatchamacallit Bars_NO_ANSWER,Q6 | White Bread_JOY,Q6 | White Bread_MEH,Q6 | White Bread_NO_ANSWER,Q6 | Whole Wheat anything_JOY,Q6 | Whole Wheat anything_MEH,Q6 | Whole Wheat anything_NO_ANSWER,Q6 | York Peppermint Patties_JOY,Q6 | York Peppermint Patties_MEH,Q6 | York Peppermint Patties_NO_ANSWER
1,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,1,0,0,1,0,...,1,0,0,1,0,0,1,0,0,1
3,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2474,1,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0
2475,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0
2476,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2477,0,0,1,0,0,1,0,0,1,0,...,1,0,0,1,0,0,1,0,0,1


**Code Check:** Make sure that the `candy`, `candy_features`, `candy_response` and `candy_features_encoded` have an index that goes from 0 to n-1 or your final CodeGrade tests will not pass.

## Final Analysis

Great work! You have now cleaned your data and prepared it to be passed to a machine learning model.  

I created models using Random Forest, Logistic Regression, and XGBoost algorithms, and they all returned around 70% accuracy rates. However, the other accuracy metrics (that you will learn about more in the machine learning classes) didn't look as good. Given the metrics that were calculated, I would say that based only on this data, using candy preference is not that great of an indicator of someone's gender.

**Next Steps:**  
- Submit your notebook to CodeGrade to check your final DataFrames.  
- For those of you that would like more practice using Pandas, try answering these optional questions below. 

## Optional Data Analysis Questions

- How many rows were in the original, uncleaned data that you imported? How many rows are in the cleaned data? How many did we end up removing from the data set?
- What percentage of respondents are planning to go out trick-or-treating? 
- What percentage of respondents 17 and younger are planning to go out for trick-or-treating?
- What are the total value counts and the normalized percentages of the age categories from the age column?
- What are the total counts and percentages for the gender column?
- What are the breakdown of counts for the country column?
- How many total respondents voted joy in candy corn and how many voted despair? Did more people vote joy or despair for candy corn?
- How many people voted joy in Reese's Peanut Butter Cups? In Snickers? Did more people vote joy for Reese's Peanut Butter Cups or for Snickers?
- How many people voted joy in Twix? In Kit Kats? Did more people vote joy for Twix or for Kit Kats?
- How many people voted joy in white bread? For whole wheat items? Did more people vote joy for white bread or whole wheat items?
- How many people voted joy for Bonkers the board game? For Bonkers the candy? Did more people vote joy for the board game or for the candy?
- How many people voted joy for a box of raisins? For the Blue-Ray DVD of the Real Housewives of Orange County Season 9? Did more people vote joy for a box of raisins or for the DVD?
- What is the favorite day of the week for the respondents (both by total counts and percentages)?
- Do more respondents see 'white and gold' or 'blue and black' for the [color of the dress](https://en.wikipedia.org/wiki/The_dress) (both total counts and percentages)?
- For those respondents that clicked on the media link (listed as Q12 columns on the survey), which link did they click on the most?
- How many males voted joy for receiving a copy of the Real Housewives of Orange County Season 9 DVD for Halloween? Females? Those 17 or younger?
- The authors tried a funny way to determine a respondent's political leaning. Note this was purely a joke and not meant to be scientific. How many total respondents voted joy in the following: Blue M&M's, Red M&M's, Green Party M&M's, Independent M&M's, and Abstained from M&M'ing?
- Determine what candy/item from the `candy_reduced` DataFrame has the most number of JOY votes and the least number of JOY votes. 
- What candy/item has the most DESPAIR votes?
- What candy/item has the most MEH votes?
- What candy/item did the most people not recognize or have an opinion on? (According to the survey, the respondents were asked to leave a question blank if they did not know the item)